In [10]:
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import sys
import os
import numpy as np
from PIL import Image   #匯入Image模組
import pandas as pd



In [2]:
DATASET_PATH  = 'image_data'
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 5
BATCH_SIZE = 8
FREEZE_LAYERS = 2
NUM_EPOCHS = 36
WEIGHTS_FINAL = 'model-resnet50-final.h5'


In [3]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)


Found 2823 images belonging to 5 classes.
Found 0 images belonging to 0 classes.


In [4]:
# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
    

Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


In [5]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 將原本的 Dense layer 拔掉，因為原本這個網路是用來做 1000 個分類的模型，我們必須替換成自己的 Dense layer 來符合我們自己資料集的類別數量
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer(全連接層)，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層（前面兩層凍結）
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [19]:
# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,                      
                        epochs = NUM_EPOCHS)



Epoch 1/20
352/352 [==============================] - 571s 2s/step - loss: 2.5420 - accuracy: 0.3897
Epoch 2/20
352/352 [==============================] - 4883s 14s/step - loss: 1.5523 - accuracy: 0.5893
Epoch 3/20
352/352 [==============================] - 5099s 14s/step - loss: 1.2524 - accuracy: 0.6870
Epoch 4/20
352/352 [==============================] - 16493s 47s/step - loss: 1.0445 - accuracy: 0.7297
Epoch 5/20
352/352 [==============================] - 1237s 4s/step - loss: 0.8763 - accuracy: 0.7726
Epoch 6/20
352/352 [==============================] - 14836s 42s/step - loss: 0.8456 - accuracy: 0.7790
Epoch 7/20
352/352 [==============================] - 1243s 4s/step - loss: 0.7115 - accuracy: 0.8053
Epoch 8/20
352/352 [==============================] - 549s 2s/step - loss: 0.6887 - accuracy: 0.8199
Epoch 9/20
352/352 [==============================] - 628s 2s/step - loss: 0.6463 - accuracy: 0.8281
Epoch 10/20
352/352 [==============================] - 552s 2s/step - loss: 0.5

In [20]:
# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)


In [6]:
def get_imlist(path):   #讀取特定資料夾下的jpg格式影象，返回圖片所在路徑的列表   
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg')]

testImages_path = get_imlist('image_data/test')
testImages_path[0]


In [42]:
ans=[]
names=[]

# 從參數讀取圖檔路徑
files = testImages_path

# 載入訓練好的模型
net = load_model('model-resnet50-epoch36.h5')

cls_list = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

# 辨識每一張圖
for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)                                        #變成四維
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]                                    #每個類別機率由大到小排序
    
    
    names.append(f[16:-4])                                                 #image檔名
    ans.append(top_inds[0])
    
    #print (top_inds[0])
    #for i in top_inds:
    #    print('    {:.3f}  {}'.format(pred[i], cls_list[i]))
    
    

In [43]:
sub = pd.DataFrame({'id': names, 'flower_class': ans})
#sub.to_csv('resnet37.csv', index=False)
sub


,id,flower_class
0,e86789078f3731bdc3d1e740825f29b3,3
1,6b0e58f46b51157485d532c8eeec6179,3
2,fd9cac41cda00325613b362025eb9cc1,1
3,10ede359e864dab7c381d9f8bb35de15,1
4,e73078a312effc5db81b849b083d1365,3
...,...,...
1995,5ba0a1dd4895ff62cd322dccbeab4d33,1
1996,39706bf6a1eb9fc681952d73cfdaa978,2
1997,09700570b778d58ab42a1aa8b62f4f83,4
1998,4cf03c70d04f9bcbf9918b896d21514f,4
